# Introduction to Convnets

This notebook is based on the code samples of Chapter 8 of [Deep Learning with Python](https://www.manning.com/books/deep-learning-with-python?a_aid=keras&a_bid=76564dff) and hosted on https://github.com/fchollet/deep-learning-with-python-notebooks.

In [1]:
import tensorflow as tf
tf.config.experimental.list_physical_devices()

2023-03-02 18:14:31.178264: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-02 18:14:32.479640: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvrtc.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/diego/anaconda3/lib/:/home/diego/anaconda3/lib/:/home/diego/anaconda3/envs/comp3420/lib/
2023-03-02 18:14:32.479800: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/diego/anaconda3/lib/:/home

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

2023-03-02 18:14:33.685701: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-02 18:14:33.686341: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [2]:
from tensorflow import keras
keras.__version__

'2.11.0'

# A Convnet for the MNIST Dataset

The following network will be used to process the MNIST dataset


In [3]:
from tensorflow.keras import layers
from tensorflow.keras import models

mnist_model = models.Sequential()
mnist_model.add(layers.Conv2D(filters=32, kernel_size=3, activation="relu", input_shape=(28, 28, 1)))
mnist_model.add(layers.MaxPooling2D(pool_size=2))
mnist_model.add(layers.Conv2D(filters=64, kernel_size=3, activation="relu"))
mnist_model.add(layers.MaxPooling2D(pool_size=2))
mnist_model.add(layers.Conv2D(filters=128, kernel_size=3, activation="relu"))
mnist_model.add(layers.Flatten())
mnist_model.add(layers.Dense(10, activation="softmax"))

2023-03-02 18:14:33.889197: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-02 18:14:33.890021: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-02 18:14:33.890994: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-02 18:14:33.891974: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least on

In [4]:
mnist_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 3, 3, 128)         73856     
                                                                 
 flatten (Flatten)           (None, 1152)              0

# Training using the MNIST Dataset

The following code will load the MNIST dataset from keras and prepare it for processing with the convnet. Note that the input to the convnet are matrices of size 28 x 28 x 1. The original images are of shape 28 x 28, so we still need to reshape them.

In [5]:
from tensorflow.keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

In [6]:
train_images.shape

(60000, 28, 28)

In [7]:
test_images.shape

(10000, 28, 28)

In [8]:
train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype("float32") / 255
test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype("float32") / 255

In [9]:
train_images.shape

(60000, 28, 28, 1)

In [10]:
test_images.shape

(10000, 28, 28, 1)

Let's compile the model and train it. To train the model, we will use 20% of the training data as validation data, which we will use to instruct the system to stop as soon as the validation data accuracy decreases.

In [11]:
mnist_model.compile(optimizer='adam',
                    loss='sparse_categorical_crossentropy',
                    metrics=['accuracy'])
history = mnist_model.fit(train_images, train_labels, epochs=10, batch_size=128, 
                          validation_split=0.2,
                          callbacks=[tf.keras.callbacks.EarlyStopping(patience=1)])

Epoch 1/10


2023-03-02 18:14:43.248622: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8100
2023-03-02 18:14:55.535345: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7fbfd6e935a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-03-02 18:14:55.535434: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce RTX 2080 Ti, Compute Capability 7.5
2023-03-02 18:14:55.752335: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-03-02 18:14:57.460423: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


375/375 [==============================] - 29s 14ms/step - loss: 0.2582 - accuracy: 0.9259 - val_loss: 0.0803 - val_accuracy: 0.9761
Epoch 2/10
375/375 [==============================] - 3s 9ms/step - loss: 0.0645 - accuracy: 0.9797 - val_loss: 0.0589 - val_accuracy: 0.9819
Epoch 3/10
375/375 [==============================] - 5s 12ms/step - loss: 0.0444 - accuracy: 0.9864 - val_loss: 0.0532 - val_accuracy: 0.9842
Epoch 4/10
375/375 [==============================] - 5s 14ms/step - loss: 0.0350 - accuracy: 0.9889 - val_loss: 0.0424 - val_accuracy: 0.9869
Epoch 5/10
375/375 [==============================] - 5s 13ms/step - loss: 0.0280 - accuracy: 0.9912 - val_loss: 0.0394 - val_accuracy: 0.9895
Epoch 6/10
375/375 [==============================] - 5s 14ms/step - loss: 0.0236 - accuracy: 0.9923 - val_loss: 0.0447 - val_accuracy: 0.9889


Let's now evaluate the test data.

In [12]:
test_loss, test_acc = mnist_model.evaluate(test_images, test_labels)
print(f"Test accuracy: {test_acc:.3f}")

313/313 [==============================] - 2s 7ms/step - loss: 0.0309 - accuracy: 0.9906
Test accuracy: 0.991


Accuracy of the test date has increased from 97.8% when we were using a simple neural network, to 99.06% with this simple convnet. A 1% increase when we are so close to 100% is a big improvement!